In [1]:
import pandas as pd
import numpy as np

path = r"C:/Users/cbi7088/NCAA2020"

In [2]:
#### Preparing Training data for tourney result ####

tourney_result = pd.read_csv(path+'/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
tourney_seed = pd.read_csv(path+'/MDataFiles_Stage1/MNCAATourneySeeds.csv')
tourney_result = tourney_result.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'WSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'LSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)

tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed
0,1985,1116,1234,X09,X08
1,1985,1120,1345,Z11,Z06
2,1985,1207,1250,W01,W16
3,1985,1229,1425,Y09,Y08
4,1985,1242,1325,Z03,Z14
...,...,...,...,...,...
2246,2019,1120,1246,Y05,Y02
2247,2019,1277,1181,W02,W01
2248,2019,1403,1277,X03,W02
2249,2019,1438,1120,Z01,Y05


In [3]:
season_result = pd.read_csv(path+'/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')

wdf = season_result.groupby(['Season','WTeamID'],as_index=False).count()
wdf = wdf.drop(['WScore','LTeamID','LScore','WLoc','NumOT'],axis=1)
wdf.rename(columns={'WTeamID':'TeamID', 'DayNum':'Wins'}, inplace=True)

ldf = season_result.groupby(['Season','LTeamID'],as_index=False).count()
ldf = ldf.drop(['LScore','WTeamID','WScore','WLoc','NumOT'],axis=1)
ldf.rename(columns={'LTeamID':'TeamID', 'DayNum':'Losses'}, inplace=True)

wl_df = pd.merge(wdf, ldf, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')
wl_df['WinPct'] = wl_df['Wins']/(wl_df['Wins']+wl_df['Losses'])
wl_df

,Season,TeamID,Wins,Losses,WinPct
0,1985,1102,5,19.0,0.208333
1,1985,1103,9,14.0,0.391304
2,1985,1104,21,9.0,0.700000
3,1985,1106,10,14.0,0.416667
4,1985,1108,19,6.0,0.760000
...,...,...,...,...,...
11222,2019,1462,18,15.0,0.545455
11223,2019,1463,21,7.0,0.750000
11224,2019,1464,10,20.0,0.333333
11225,2019,1465,12,14.0,0.461538


In [4]:

# Add WinPct to the tourney result and create WinPctDiff

tourney_result = pd.merge(tourney_result, wl_df, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Wins':'WTWins', 'Losses':'WTLosses', 'WinPct':'WTWinPct'}, inplace=True)
tourney_result = pd.merge(tourney_result, wl_df, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Wins':'LTWins', 'Losses':'LTLosses', 'WinPct':'LTWinPct'}, inplace=True)
tourney_result = tourney_result.drop(['TeamID_x','TeamID_y'],axis=1)
tourney_result['WinPctDiff'] = tourney_result['WTWinPct']-tourney_result['LTWinPct']
tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed,WTWins,WTLosses,WTWinPct,LTWins,LTLosses,LTWinPct,WinPctDiff
0,1985,1116,1234,X09,X08,21,12.0,0.636364,20,10.0,0.666667,-0.030303
1,1985,1120,1345,Z11,Z06,18,11.0,0.620690,17,8.0,0.680000,-0.059310
2,1985,1207,1250,W01,W16,25,2.0,0.925926,11,18.0,0.379310,0.546616
3,1985,1229,1425,Y09,Y08,20,7.0,0.740741,19,9.0,0.678571,0.062169
4,1985,1242,1325,Z03,Z14,23,7.0,0.766667,20,7.0,0.740741,0.025926
...,...,...,...,...,...,...,...,...,...,...,...,...
2246,2019,1120,1246,Y05,Y02,25,9.0,0.735294,27,6.0,0.818182,-0.082888
2247,2019,1277,1181,W02,W01,28,6.0,0.823529,29,5.0,0.852941,-0.029412
2248,2019,1403,1277,X03,W02,26,6.0,0.812500,28,6.0,0.823529,-0.011029
2249,2019,1438,1120,Z01,Y05,29,3.0,0.906250,25,9.0,0.735294,0.170956


In [5]:
#### Preparing training data to make seeds int ####

def get_seed(x):
    return int(x[1:3])

tourney_result['WSeed'] = tourney_result['WSeed'].map(lambda x: get_seed(x))
tourney_result['LSeed'] = tourney_result['LSeed'].map(lambda x: get_seed(x))

In [6]:
#### Preparing training data ####
#Split season result into df for all game wins, df for all game losses, then concat dfs to show total points scored in season

season_win_result = season_result[['Season', 'WTeamID', 'WScore']]
season_lose_result = season_result[['Season', 'LTeamID', 'LScore']]
season_win_result.rename(columns={'WTeamID':'TeamID', 'WScore':'Score'}, inplace=True)
season_lose_result.rename(columns={'LTeamID':'TeamID', 'LScore':'Score'}, inplace=True)
season_result = pd.concat((season_win_result, season_lose_result)).reset_index(drop=True)
season_score = season_result.groupby(['Season', 'TeamID'])['Score'].sum().reset_index()

C:\Users\cbi7088\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [7]:
season_score

,Season,TeamID,Score
0,1985,1102,1514
1,1985,1103,1404
2,1985,1104,2055
3,1985,1106,1719
4,1985,1108,2075
...,...,...,...
11236,2019,1462,2372
11237,2019,1463,2265
11238,2019,1464,2205
11239,2019,1465,1962


In [8]:
season_score = pd.merge(season_score, wl_df, left_on=['Season','TeamID'], right_on=['Season','TeamID'], how='inner')
season_score

,Season,TeamID,Score,Wins,Losses,WinPct
0,1985,1102,1514,5,19.0,0.208333
1,1985,1103,1404,9,14.0,0.391304
2,1985,1104,2055,21,9.0,0.700000
3,1985,1106,1719,10,14.0,0.416667
4,1985,1108,2075,19,6.0,0.760000
...,...,...,...,...,...,...
11222,2019,1462,2372,18,15.0,0.545455
11223,2019,1463,2265,21,7.0,0.750000
11224,2019,1464,2205,10,20.0,0.333333
11225,2019,1465,1962,12,14.0,0.461538


In [9]:
#### Preparing training data ####
# Merging tourney result df with season score df

tourney_result = pd.merge(tourney_result, season_score, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Score':'WScoreT'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, season_score, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Score':'LScoreT'}, inplace=True)
tourney_result = tourney_result.drop(['TeamID','Wins_y','Losses_y','WinPct_y'], axis=1)

where_are_NaNs = np.isnan(tourney_result)
tourney_result[where_are_NaNs] = .01

tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed,WTWins,WTLosses,WTWinPct,LTWins,LTLosses,LTWinPct,WinPctDiff,WScoreT,Wins_x,Losses_x,WinPct_x,LScoreT
0,1985,1116,1234,9,8,21,12.0,0.636364,20,10.0,0.666667,-0.030303,2156,21,12.0,0.636364,2092
1,1985,1120,1345,11,6,18,11.0,0.620690,17,8.0,0.680000,-0.059310,2040,18,11.0,0.620690,1728
2,1985,1207,1250,1,16,25,2.0,0.925926,11,18.0,0.379310,0.546616,2045,25,2.0,0.925926,1907
3,1985,1229,1425,9,8,20,7.0,0.740741,19,9.0,0.678571,0.062169,1933,20,7.0,0.740741,1915
4,1985,1242,1325,3,14,23,7.0,0.766667,20,7.0,0.740741,0.025926,2281,23,7.0,0.766667,1824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2246,2019,1120,1246,5,2,25,9.0,0.735294,27,6.0,0.818182,-0.082888,2682,25,9.0,0.735294,2532
2247,2019,1277,1181,2,1,28,6.0,0.823529,29,5.0,0.852941,-0.029412,2680,28,6.0,0.823529,2839
2248,2019,1403,1277,3,2,26,6.0,0.812500,28,6.0,0.823529,-0.011029,2339,26,6.0,0.812500,2680
2249,2019,1438,1120,1,5,29,3.0,0.906250,25,9.0,0.735294,0.170956,2299,29,3.0,0.906250,2682


In [10]:
#### Preparing training data ####
# New df containing Seed Diff, Score Diff, and tourney game result
# Also concats game data into winning and losing (doubling amount of data)

tourney_win_result = tourney_result.drop(['Season', 'WTeamID', 'LTeamID','Wins_x','Losses_x','WinPct_x'], axis=1)
tourney_win_result.rename(columns={'WSeed':'Seed1', 'LSeed':'Seed2', 'WScoreT':'ScoreT1', 'LScoreT':'ScoreT2'}, inplace=True)

tourney_lose_result = tourney_win_result.copy()
tourney_lose_result['Seed1'] = tourney_win_result['Seed2']
tourney_lose_result['Seed2'] = tourney_win_result['Seed1']
tourney_lose_result['ScoreT1'] = tourney_win_result['ScoreT2']
tourney_lose_result['ScoreT2'] = tourney_win_result['ScoreT1']

tourney_win_result['Seed_diff'] = tourney_win_result['Seed1'] - tourney_win_result['Seed2']
tourney_win_result['ScoreT_diff'] = tourney_win_result['ScoreT1'] - tourney_win_result['ScoreT2']
tourney_lose_result['Seed_diff'] = tourney_lose_result['Seed1'] - tourney_lose_result['Seed2']
tourney_lose_result['ScoreT_diff'] = tourney_lose_result['ScoreT1'] - tourney_lose_result['ScoreT2']

tourney_win_result['result'] = 1
tourney_lose_result['result'] = 0
tourney_result = pd.concat((tourney_win_result, tourney_lose_result)).reset_index(drop=True)

tourney_result

,Seed1,Seed2,WTWins,WTLosses,WTWinPct,LTWins,LTLosses,LTWinPct,WinPctDiff,ScoreT1,ScoreT2,Seed_diff,ScoreT_diff,result
0,9,8,21,12.0,0.636364,20,10.0,0.666667,-0.030303,2156,2092,1,64,1
1,11,6,18,11.0,0.620690,17,8.0,0.680000,-0.059310,2040,1728,5,312,1
2,1,16,25,2.0,0.925926,11,18.0,0.379310,0.546616,2045,1907,-15,138,1
3,9,8,20,7.0,0.740741,19,9.0,0.678571,0.062169,1933,1915,1,18,1
4,3,14,23,7.0,0.766667,20,7.0,0.740741,0.025926,2281,1824,-11,457,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4497,2,5,25,9.0,0.735294,27,6.0,0.818182,-0.082888,2532,2682,-3,-150,0
4498,1,2,28,6.0,0.823529,29,5.0,0.852941,-0.029412,2839,2680,-1,159,0
4499,2,3,26,6.0,0.812500,28,6.0,0.823529,-0.011029,2680,2339,-1,341,0
4500,5,1,29,3.0,0.906250,25,9.0,0.735294,0.170956,2682,2299,4,383,0


In [11]:
#### Create model ####

import matplotlib.pyplot as plt  

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

In [12]:
#### Create model ####
# Create train and test set from training data

scaler = StandardScaler()
#scaled_data = scaler.fit_transform(tourney_result)

X = tourney_result.drop('result', axis=1)
y = tourney_result.result

X = scaler.fit_transform(X)

seed = 37745

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= seed)

In [13]:
tourney_result

,Seed1,Seed2,WTWins,WTLosses,WTWinPct,LTWins,LTLosses,LTWinPct,WinPctDiff,ScoreT1,ScoreT2,Seed_diff,ScoreT_diff,result
0,9,8,21,12.0,0.636364,20,10.0,0.666667,-0.030303,2156,2092,1,64,1
1,11,6,18,11.0,0.620690,17,8.0,0.680000,-0.059310,2040,1728,5,312,1
2,1,16,25,2.0,0.925926,11,18.0,0.379310,0.546616,2045,1907,-15,138,1
3,9,8,20,7.0,0.740741,19,9.0,0.678571,0.062169,1933,1915,1,18,1
4,3,14,23,7.0,0.766667,20,7.0,0.740741,0.025926,2281,1824,-11,457,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4497,2,5,25,9.0,0.735294,27,6.0,0.818182,-0.082888,2532,2682,-3,-150,0
4498,1,2,28,6.0,0.823529,29,5.0,0.852941,-0.029412,2839,2680,-1,159,0
4499,2,3,26,6.0,0.812500,28,6.0,0.823529,-0.011029,2680,2339,-1,341,0
4500,5,1,29,3.0,0.906250,25,9.0,0.735294,0.170956,2682,2299,4,383,0


In [14]:
#### Create model ####
# Pick a model

model = LogisticRegression(random_state=seed)
#model = RandomForestRegressor(random_state = seed)

model.fit(X_train, y_train)
#predictions = model.predict_proba(X_test)[:, 1]
predictions = model.predict(X_test)

In [15]:
#### Create model ####

print('X_train shape: ',X_train.shape)
print('y_train shape: ',y_train.shape)
print('X_test shape: ',X_test.shape)
print('y_test shape: ',y_test.shape)
print('predictions shape: ', predictions.shape)
print('Model Score: ', model.score(X_test, y_test))
print(model.coef_)

print(tourney_result.columns)

X_train shape:  (3601, 13)
y_train shape:  (3601,)
X_test shape:  (901, 13)
y_test shape:  (901,)
predictions shape:  (901,)
Model Score:  0.7103218645948945
[[-3.81848707e-01  3.98151175e-01 -4.11012048e-02 -9.19062598e-02
  -6.53090373e-05 -7.45063309e-03  1.12891237e-02 -6.91514852e-02
  -1.18287568e-01  1.13955203e-01 -8.19411536e-02 -4.78790814e-01
   1.43649261e-01]]
Index(['Seed1', 'Seed2', 'WTWins', 'WTLosses', 'WTWinPct', 'LTWins',
       'LTLosses', 'LTWinPct', 'WinPctDiff', 'ScoreT1', 'ScoreT2', 'Seed_diff',
       'ScoreT_diff', 'result'],
      dtype='object')


In [16]:
t_list = tourney_result.columns
t_list = t_list[:-1]
t_list

Index(['Seed1', 'Seed2', 'WTWins', 'WTLosses', 'WTWinPct', 'LTWins',
       'LTLosses', 'LTWinPct', 'WinPctDiff', 'ScoreT1', 'ScoreT2', 'Seed_diff',
       'ScoreT_diff'],
      dtype='object')

In [17]:
dfObj = pd.DataFrame(model.coef_, columns = t_list).T.abs().sort_values(by=0, ascending=False)
dfObj

,0
Seed_diff,0.478791
Seed2,0.398151
Seed1,0.381849
ScoreT_diff,0.143649
WinPctDiff,0.118288
ScoreT1,0.113955
WTLosses,0.091906
ScoreT2,0.081941
LTWinPct,0.069151
WTWins,0.041101


In [18]:
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

kfold = model_selection.KFold(n_splits=10, random_state=seed)
scoring = 'accuracy'
results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring)
print('CV scoring: ', 'Mean: ', results.mean().round(3), 'Stdev: ', results.std().round(3))

matrix = confusion_matrix(y_test, predictions)
print(matrix)

report = classification_report(y_test, predictions,digits=3)
print(report)

CV scoring:  Mean:  0.58 Stdev:  0.037
[[321 134]
 [127 319]]
              precision    recall  f1-score   support

           0      0.717     0.705     0.711       455
           1      0.704     0.715     0.710       446

    accuracy                          0.710       901
   macro avg      0.710     0.710     0.710       901
weighted avg      0.710     0.710     0.710       901

